In [25]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer # Feature Engineering
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [26]:
# global variables
features = ["Age", "Gender", "Department", "DailyRate"] # to be determined
target = "Attrition"


In [27]:
def main():
    # create dataframe with project's data set
    df = pd.read_csv('HR.csv')
    # clean the data
    clean_df = clean_data(df)
    # get features matrix X and target variable y
    X, y = feature_selection(clean_df)
    

In [49]:
def clean_data(df):
    # drop columns that are not needed
    for col_header in df.columns:
        if col_header not in features and col_header != target:
            df.drop(col_header, axis = 1, inplace=True)
    # drop rows that contain at least one empty cell
    df.dropna(axis='rows')
    
    # feature enigneering for categorical data
    # feature engineering for text data
    
    return df


In [33]:
def feature_selection(df):
    # split dataset in features and target variable
    X = df[features] # Features Matrix
    y = df[target] # Target variable
    return X, y


In [50]:
# test
df = pd.read_csv('HR.csv')
print(clean_data(df))



      Age Attrition  DailyRate              Department  Gender
0      41       Yes       1102                   Sales  Female
1      49        No        279  Research & Development    Male
2      37       Yes       1373  Research & Development    Male
3      33        No       1392  Research & Development  Female
4      27        No        591  Research & Development    Male
...   ...       ...        ...                     ...     ...
1465   36        No        884  Research & Development    Male
1466   39        No        613  Research & Development    Male
1467   27        No        155  Research & Development    Male
1468   49        No       1023                   Sales    Male
1469   34        No        628  Research & Development    Male

[1470 rows x 5 columns]
